In [ ]:
pip install pandas googlemaps openpyxl tqdm

In [ ]:
import pandas as pd
import googlemaps
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()  # Enables progress_apply for pandas

In [ ]:
# Read API key from the file
with open('GMapAPI.txt', 'r') as file:
    api_key = file.read().strip()
    
# Load data from the Excel file
df = pd.read_excel('Tableau Project Data.xlsx', sheet_name='listings')

# Initialize Google Maps client
gmaps = googlemaps.Client(key=api_key)

# Add 'Country' and 'Province' columns
df['Country'] = 'Canada'
df['Province'] = 'BC'

In [ ]:
# Function to retrieve postal code from coordinates
def get_postal_code(lat, lon):
    try:
        reverse_geocode_result = gmaps.reverse_geocode((lat, lon))
        for component in reverse_geocode_result[0]['address_components']:
            if 'postal_code' in component['types']:
                return component['long_name']
    except Exception as e:
        print(f"Error during reverse geocoding: {e}")
        return "Not Found"  # Returns 'Not Found' if postal code is not available

In [ ]:
# Apply function to dataframe with progress bar
df['Postal Code'] = df.progress_apply(lambda row: get_postal_code(row['latitude'], row['longitude']), axis=1)

In [ ]:
na_postal_codes_count = df['Postal Code'].isna().sum()
print(f"Number of rows with NA postal code: {na_postal_codes_count}")

In [ ]:
# Save the modified dataframe back to Excel
df.to_excel('Updated_Tableau_Project_Data.xlsx', index=False)